## Notas para la práctica

Lo que tenemos que hacer es entender el ejemplo, e intentar mejoraro. Por mejorarlo entiendo, por ejemplo, lo siguiente:
   - cambiar la CNN de VGG16 a otro tipo
   - encontrar buenos parámetros (no hacer un grid seach, recomendacion del profe)
   - jugar con la LSTM a ver si conseguimos que de mejores resultados... 
   - Como guinda al pastel, si utilizamos otro dataset diferente, pues también estaría bien. 
   
PROFE:
Como veo que hay algunas dudas con respecto a la práctica, os comento por aquí lo que espero que hagáis.El ejemplo que os proporciono es tremendamente sencillo y, como podéis comprobar, no funciona ni mucho menos de una forma aceptable. Así que una cosa que podéis hacer es seguir con ese ejemplo y cambiar el dataset, jugar con los parámetros (no recomendaría un grid search, acordaos de que hay formas mejores de hacerlo :wink: ), etc.Ahora bien, lo que yo os aconsejo que hagáis es que miréis en los ejemplos que os adjunté al final del notebook (u otros que encontréis vosotros) donde se explican con ejemplos diferentes arquitecturas y enfoques, que escojáis uno de ellos y que tratéis de hacerlo funcionar. Eso lo primero, y después, que miréis otros ejemplos para combinarlos con lo que ya tenéis e intentar mejorar lo que conseguís. 
   
PROFE 2: 
Según podéis ver, las descripciones que damos de nuestras imágenes no son las más precisas del mundo. Esto es porque este ejemplo usa muy pocos datos, en la FUENTE que tenéis más abajo, que es de donde he extraído este ejemplo, podéis ver cómo mejoran lo que hemos hecho aquí.

FUENTE: https://machinelearningmastery.com/develop-a-caption-generation-model-in-keras/

Os dejo también varios enlaces que podéis consultar para conseguir hacer algo mejor todavía:

    https://machinelearningmastery.com/how-to-caption-photos-with-deep-learning/
    https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/
    https://daniel.lasiman.com/post/image-captioning/
    https://www.oreilly.com/learning/caption-this-with-tensorflow
    https://towardsdatascience.com/image-captioning-in-deep-learning-9cd23fb4d8d2
    https://deeplearningmania.quora.com/Keras-deep-learning-for-image-caption-retrieval

Todos ellos son muy didácticos y os van a servir de mucho a la hora de hacer la práctica. Lamento no poder proporcionaros material en español, pero desgraciadamente este mundo evoluciona muy rápido y lo mejor está siempre en inglés. Lo único que puedo deciros es que estamos trabajando en solucionar esto, y en unos meses esperamos tener un blog donde proporcionar ejemplos de alto nivel, de forma didáctica y a la vez compleja, en un perfecto castellano manchego. Si estáis interesados decidmelo y en cuanto lo lancemos os lo haremos saber!


## Descarga del fichero flickr30k con las descripciones de las imágnes

In [2]:
!wget http://shannon.cs.illinois.edu/DenotationGraph/data/flickr30k.tar.gz
 

--2018-06-28 18:24:09--  http://shannon.cs.illinois.edu/DenotationGraph/data/flickr30k.tar.gz
Resolving shannon.cs.illinois.edu (shannon.cs.illinois.edu)... 192.17.90.133
Connecting to shannon.cs.illinois.edu (shannon.cs.illinois.edu)|192.17.90.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3652513 (3.5M) [application/x-gzip]
Saving to: ‘flickr30k.tar.gz’

flickr30k.tar.gz    100%[===================>]   3.48M  --.-KB/s    in 0.1s    

2018-06-28 18:24:09 (30.8 MB/s) - ‘flickr30k.tar.gz’ saved [3652513/3652513]



In [3]:
!ls -la

total 3604
drwxr-xr-x 1 root root    4096 Jun 28 18:24 .
drwxr-xr-x 1 root root    4096 Jun 28 18:22 ..
drwx------ 4 root root    4096 Jun 28 18:23 .cache
drwxr-xr-x 3 root root    4096 Jun 28 18:23 .config
drwxr-xr-x 3 root root    4096 Jun 25 16:59 datalab
-rw-r--r-- 1 root root 3652513 Sep 21  2015 flickr30k.tar.gz
drwxr-xr-x 4 root root    4096 Jun 28 18:23 .forever
drwxr-xr-x 5 root root    4096 Jun 28 18:23 .ipython
drwx------ 3 root root    4096 Jun 28 18:23 .local


In [4]:
!tar -tvf flickr30k.tar.gz

-rw-r--r-- pyoung2/14987   682 2013-12-24 09:29 readme.txt
-rw-r--r-- pyoung2/pyoung2 13023677 2013-12-16 18:20 results_20130124.token


In [5]:
!tar -zxvf flickr30k.tar.gz

readme.txt
results_20130124.token


In [6]:
!grep 405961988 results_20130124.token

405961988.jpg#0	The construction worker is drilling into the street near a street with active traffic .
405961988.jpg#1	The man in the construction gear uses a large , red drill .
405961988.jpg#2	A man wearing a hi-viz jacket is drilling into the road .
405961988.jpg#3	A construction worker drills a hole into the ground .
405961988.jpg#4	A man wearing a yellow vest works in the street .


## Prepararmos los datos mediante las funciones axiliares faciliadas en el curso. 
*Mi "aportación" en este punto ha  sido entender el código por completo y documentar aquellas funciones que desconocía:*

In [0]:
# Carga el contenido de un fichero en una variable para trabajar con ella
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
filename = 'results_20130124.token'
# load descriptions
doc = load_doc(filename)

In [8]:
print("Datos relativos al fichero " + filename + " cargados en 'doc'")
print('Tipo de fichero: %s ' % type(doc))
print('Longitud del fichero: %d ' % len(doc))
print("200 primeros carcteres:")
doc[:200].split('\n')

Datos relativos al fichero results_20130124.token cargados en 'doc'
Tipo de fichero: <class 'str'> 
Longitud del fichero: 13023667 
200 primeros carcteres:


['1000092795.jpg#0\tTwo young guys with shaggy hair look at their hands while hanging out in the yard .',
 '1000092795.jpg#1\tTwo young , White males are outside near many bushes .',
 '1000092795.jpg#2\tTwo men in']

In [9]:
# extract descriptions for images
# sólo se almacenará la primera descripción de cada imagen
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
    # Si la linea tiene 2 o menos caracteres se descarta y se selecciona la siguiente
		if len(line) < 2:
			continue
		# take the first token as the image id (tokens[0]), the rest as the description (tokens[1:])
		image_id, image_desc = tokens[0], tokens[1:]
		# remove extension from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# store the first description for each image in a dictionary
		if image_id not in mapping:
			mapping[image_id] = image_desc
	return mapping
 
# parse descriptions
descriptions = load_descriptions(doc)
print('Descripcion de la imagen "1000092795": %s' % descriptions['1000092795'])
print('Loaded: %d ' % len(descriptions))

Descripcion de la imagen "1000092795": Two young guys with shaggy hair look at their hands while hanging out in the yard .
Loaded: 31783 


Para "limpiar" los mensajes se convierten todas las palabras a minúsculas, se quitan los signos de puntuación así como las palabras de 1 solo caracter

In [10]:
import string

def clean_descriptions(descriptions):
  
  # str.maketrans('', '', string.punctuation)
  # This uses the 3-argument version of str.maketrans
  # with arguments (x, y, z) where 'x' and 'y'
  # must be equal-length strings and characters in 'x'
  # are replaced by characters in 'y'. 'z'
  # is a string (string.punctuation here)
  # where each character in the string is mapped to None

	# prepare translation table for removing punctuation	
  table = str.maketrans('', '', string.punctuation)
  # descriptions.items() son los distintos elementos del diccionario
  for key, desc in descriptions.items():
		# tokenize: pasa el string de la descripción a una lista de palabras
    desc = desc.split()
		# convert to lower case
    desc = [word.lower() for word in desc]
		# remove punctuation from each token: Aquí aplicamos la tabla anterior
    desc = [w.translate(table) for w in desc]
		# remove hanging 's' and 'a'
    desc = [word for word in desc if len(word)>1]
		# store as string
    descriptions[key] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary: número de palabras
all_tokens = ' '.join(descriptions.values()).split()
vocabulary = set(all_tokens)
print('Descripcion de la imagen "1000092795": %s' % descriptions['1000092795'])
print('Vocabulary Size: %d' % len(vocabulary))

Descripcion de la imagen "1000092795": two young guys with shaggy hair look at their hands while hanging out in the yard
Vocabulary Size: 13130


Guardamos el dataset modificado

In [0]:
# save descriptions to file, one per line
def save_doc(descriptions, filename):
	lines = list()
	for key, desc in descriptions.items():
		lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# save descriptions
save_doc(descriptions, 'descriptions.txt')

In [12]:
print("Datos relativos al fichero " + filename + " cargados en 'doc'")
print('Tipo de fichero: %s ' % type(descriptions))
print('Longitud del fichero: %d ' % len(descriptions))
print('Descripcion de la imagen "1000092795": %s' % descriptions['1000092795'])


Datos relativos al fichero results_20130124.token cargados en 'doc'
Tipo de fichero: <class 'dict'> 
Longitud del fichero: 31783 
Descripcion de la imagen "1000092795": two young guys with shaggy hair look at their hands while hanging out in the yard


# Preparación de las imágenes

In [1]:
!wget http://shannon.cs.illinois.edu/DenotationGraph/data/flickr30k-images.tar

--2018-06-30 09:43:20--  http://shannon.cs.illinois.edu/DenotationGraph/data/flickr30k-images.tar
Resolving shannon.cs.illinois.edu (shannon.cs.illinois.edu)... 192.17.90.133
Connecting to shannon.cs.illinois.edu (shannon.cs.illinois.edu)|192.17.90.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4440780800 (4.1G) [application/x-tar]
Saving to: ‘flickr30k-images.tar’

flickr30k-images.ta  85%[================>   ]   3.53G  87.9MB/s    eta 8s     

flickr30k-images.ta 100%[===================>]   4.14G  89.0MB/s    in 49s     

2018-06-30 09:44:09 (85.7 MB/s) - ‘flickr30k-images.tar’ saved [4440780800/4440780800]



In [0]:
!tar -xf flickr30k-images.tar

In [3]:
!ls -la

total 4337780
drwxr-xr-x 1 root root        4096 Jun 30 09:46 .
drwxr-xr-x 1 root root        4096 Jun 30 09:38 ..
drwx------ 4 root root        4096 Jun 30 09:42 .cache
drwxr-xr-x 3 root root        4096 Jun 30 09:42 .config
drwxr-xr-x 3 root root        4096 Jun 28 16:55 datalab
drwxr-xr-x 2  501 staff    1060864 Jan 15  2014 flickr30k-images
-rw-r--r-- 1 root root  4440780800 Sep 21  2015 flickr30k-images.tar
drwxr-xr-x 4 root root        4096 Jun 30 09:39 .forever
drwxr-xr-x 5 root root        4096 Jun 30 09:42 .ipython
drwx------ 3 root root        4096 Jun 30 09:39 .local


In [4]:
# https://pypi.org/project/tqdm/
# tqdm means “progress” in Arabic, Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you’re done!
!pip install tqdm

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [5]:
from os import listdir
from pickle import load, dump
from tqdm import tqdm
from numpy import array
from numpy import argmax
from pandas import DataFrame
from nltk.translate.bleu_score import corpus_bleu
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Input, Dense, Flatten, LSTM, RepeatVector, TimeDistributed, Embedding
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D

Using TensorFlow backend.


In [6]:
!ls flickr30k-images/|grep readme
!rm flickr30k-images/readme.txt

readme.txt


In [7]:
# extract features from each photo in the directory
def extract_features(directory):
  # load the model
  in_layer = Input(shape=(224, 224, 3))
  model = VGG16(include_top=False, input_tensor=in_layer)
  print(model.summary())
  # extract features from each photo
  features = dict()
  files_in_directory = listdir(directory)
  n_images = len(files_in_directory)
  for i, name in tqdm(enumerate(files_in_directory)):
    # load an image from file
    filename = directory + '/' + name
    image = load_img(filename, target_size=(224, 224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    # get image id
    image_id = name.split('.')[0]
    # store feature
    features[image_id] = feature
    # print('{} / {} > {}'.format(i, n_images, name))
  return features

# extract features from all images
directory = 'flickr30k-images'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
# dump(features, open('features.pkl', 'wb'))

58892288/58889256 [==============================] - 1s 0us/step


0it [00:00, ?it/s]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

31783it [18:09, 29.17it/s]

Extracted Features: 31783


In [1]:
!ls -la

total 32
drwxr-xr-x 1 root root 4096 Jun 28 22:02 .
drwxr-xr-x 1 root root 4096 Jun 28 21:30 ..
drwx------ 4 root root 4096 Jun 28 22:02 .cache
drwxr-xr-x 3 root root 4096 Jun 28 22:02 .config
drwxr-xr-x 3 root root 4096 Jun 25 16:59 datalab
drwxr-xr-x 4 root root 4096 Jun 28 21:31 .forever
drwxr-xr-x 5 root root 4096 Jun 28 22:02 .ipython
drwx------ 3 root root 4096 Jun 28 21:31 .local


In [0]:
from pickle import load

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# split a dataset into train/test elements
def train_test_split(dataset):
	# order keys so the split is consistent
	ordered = sorted(dataset)
	# return split dataset as two new sets
	return set(ordered[:100]), set(ordered[100:200])

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# store
			descriptions[image_id] = 'startseq ' + ' '.join(image_desc) + ' endseq'
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# load dev set
filename = 'Flickr_8k.devImages.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
print('Train=%d, Test=%d' % (len(train), len(test)))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))

In [0]:
# Ahora codificamos nuestras descripciones a números
from keras.preprocessing.text import Tokenizer

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = list(descriptions.values())
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

For example, the input sequence “little girl running in field” would be split into 6 input-output pairs to train the model:

X1, X2 (text sequence), y (word)

photo startseq, little

photo startseq, little, girl

photo startseq, little, girl, running

photo startseq, little, girl, running, in

photo startseq, little, girl, running, in, field

photo startseq, little, girl, running, in, field, endseq


In [0]:
# Y creamos las secuencias:

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, desc, image, max_length):
	Ximages, XSeq, y = list(), list(),list()
	vocab_size = len(tokenizer.word_index) + 1
	# integer encode the description
	seq = tokenizer.texts_to_sequences([desc])[0]
	# split one sequence into multiple X,y pairs
	for i in range(1, len(seq)):
		# select
		in_seq, out_seq = seq[:i], seq[i]
		# pad input sequence
		in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
		# encode output sequence
		out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
		# store
		Ximages.append(image)
		XSeq.append(in_seq)
		y.append(out_seq)
	# Ximages, XSeq, y = array(Ximages), array(XSeq), array(y)
	return [Ximages, XSeq, y]

In [0]:
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor (encoder)
	inputs1 = Input(shape=(7, 7, 512))
	fe1 = GlobalMaxPooling2D()(inputs1)
	fe2 = Dense(128, activation='relu')(fe1)
	fe3 = RepeatVector(max_length)(fe2)
	# embedding
	inputs2 = Input(shape=(max_length,))
	emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
	emb3 = LSTM(256, return_sequences=True)(emb2)
	emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
	# merge inputs
	merged = concatenate([fe3, emb4])
	# language model (decoder)
	lm2 = LSTM(500)(merged)
	lm3 = Dense(500, activation='relu')(lm2)
	outputs = Dense(vocab_size, activation='softmax')(lm3)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	print(model.summary())
	return model

In [0]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length, n_step):
	# loop until we finish training
	while 1:
		# loop over photo identifiers in the dataset
		keys = list(descriptions.keys())
		for i in range(0, len(keys), n_step):
			Ximages, XSeq, y = list(), list(),list()
			for j in range(i, min(len(keys), i+n_step)):
				image_id = keys[j]
				# retrieve photo feature input
				image = features[image_id][0]
				# retrieve text input
				desc = descriptions[image_id]
				# generate input-output pairs
				in_img, in_seq, out_word = create_sequences(tokenizer, desc, image, max_length)
				for k in range(len(in_img)):
					Ximages.append(in_img[k])
					XSeq.append(in_seq[k])
					y.append(out_word[k])
			# yield this batch of samples to the model
			yield [[array(Ximages), array(XSeq)], array(y)]

In [0]:
# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

Como hemos entrenado con datasets muy pequeños, nuestro dataset tiene gran variabilidad. Con lo cual, vamos a ejecutar 3 rondas de experimentos (n_repeats) y a promediar las métricas para poder dar una métrica más fiable.

In [0]:
# load dev set
filename = 'Flickr_8k.devImages.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features('features.pkl', train)
test_features = load_photo_features('features.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max(len(s.split()) for s in list(train_descriptions.values()))
print('Description Length: %d' % max_length)
 
# define experiment
model_name = 'baseline1'
verbose = 1
n_epochs = 50
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3
 
# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
	# define the model
	model = define_model(vocab_size, max_length)
	# fit model
	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	# evaluate model on training data
	train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
	test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
	# store
	train_results.append(train_score)
	test_results.append(test_score)
	print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)

Fijaos que nos da unas pérdidas medias son 0.02 en train y 0.05 en test. La verdad es que esto no nos dice mucho, así que vamos a probar a ejecutarlo viendo realmente lo que predice nuestra red LSTM:

In [0]:
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		actual.append([desc.split()])
		predicted.append(yhat.split())
		print('Actual:    %s' % desc)
		print('Predicted: %s' % yhat)
		if len(actual) >= 5:
			break
	# calculate BLEU score
	bleu = corpus_bleu(actual, predicted)
	return bleu

# define experiment
model_name = 'baseline1'
verbose = 2
n_epochs = 50
n_photos_per_update = 2
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3
 
# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
	# define the model
	model = define_model(vocab_size, max_length)
	# fit model
	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
	# evaluate model on training data
	train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
	test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
	# store
	train_results.append(train_score)
	test_results.append(test_score)
	print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)